In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 26.6 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import IPython

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import model_selection

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow import keras
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import keras_tuner as kt

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train4.csv to train4.csv


In [ ]:
train = pd.read_csv('train4.csv',encoding='utf-8')

In [ ]:
train.head()

,Unnamed: 0,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,Accountants,2.0,6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,26.0,2.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24823 entries, 0 to 24822
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     24823 non-null  int64  
 1   gender         24823 non-null  object 
 2   car            24823 non-null  object 
 3   reality        24823 non-null  object 
 4   child_num      24823 non-null  int64  
 5   income_total   24823 non-null  float64
 6   income_type    24823 non-null  object 
 7   edu_type       24823 non-null  object 
 8   family_type    24823 non-null  object 
 9   house_type     24823 non-null  object 
 10  DAYS_BIRTH     24823 non-null  int64  
 11  DAYS_EMPLOYED  24823 non-null  int64  
 12  work_phone     24823 non-null  int64  
 13  phone          24823 non-null  int64  
 14  email          24823 non-null  int64  
 15  occyp_type     24823 non-null  object 
 16  family_size    24823 non-null  float64
 17  begin_month    24823 non-null  float64
 18  credit

In [ ]:
#train4.csv 결측치행, index 열 제거
train.dropna(inplace=True)
train.drop(['Unnamed: 0'], axis=1, inplace=True)

# 성별 분류
train['gender'] = train['gender'].replace(['F','M'],[0,1])

# 차량 소유 분류
train['car'] = train['car'].replace(['N','Y'],[0,1])

# 부동산(주택) 소유 분류
train['reality'] = train['reality'].replace(['N','Y'],[0,1])
train['family_type'] = train['family_type'].replace(['Married','Civil marriage','Single / not married','Separated','Widow'],[1,1,0,0,0])

train.loc[train['family_size'] >= 5,'family_size']=5
#train.loc[train['child_num'] >= 3,'child_num']=3
#child_num 제거 (다중공선성)
#train.drop(['child_num'],axis=1,inplace=True)
train.loc[train['child_num'] >=1,'child_num']=1
train.loc[train['child_num'] <= 0,'child_num']=0

train['age'] = train['DAYS_BIRTH']*(-1)/365
train['age'] = train['age'].round(0)

# train['begin_month'] = train['begin_month']* -1
# train.loc[train['DAYS_EMPLOYED'] >0,'DAYS_EMPLOYED']=0
# train['DAYS_EMPLOYED'] = train['DAYS_EMPLOYED']* -1
train.drop('DAYS_BIRTH',axis=1,inplace=True)

train['age']= -train['age']
# test['age']= -test['age']
train['income_total'] = train['income_total']/10000 


train_target = train[['credit']].copy()
train_data = train.drop('credit',axis=1).copy()



In [ ]:
train.head(2)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,20.25,Commercial associate,Higher education,1,Municipal apartment,4709,0,0,0,Accountants,2.0,6.0,1.0,38.0
1,0,0,1,1,24.75,Commercial associate,Secondary / secondary special,1,House / apartment,1540,0,0,1,Laborers,3.0,5.0,1.0,31.0


In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_data, 
                                                                    train_target, 
                                                                    test_size=0.3, 
                                                                    random_state=0)

In [ ]:
numeric_features = ['income_total','DAYS_EMPLOYED','family_size', 'begin_month', 'age'] 
numeric_transformer = StandardScaler() # cf) RobustScaler

categorical_features = ['child_num','gender', 'car', 'reality','income_type','edu_type', 'family_type', 'house_type','work_phone','phone', 'email', 'occyp_type']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') # categories='auto' : just for ignoring warning messages

preprocessor = ColumnTransformer(
    transformers=[ # List of (name, transformer, column(s))
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
x_train_transformed = preprocessor_pipe.fit_transform(x_train)
x_test_transformed = preprocessor_pipe.transform(x_test)

x_train_transformed

array([[ 0.25114229,  0.05136773,  0.89937497, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.20577259, -0.92394361, -0.22023669, ...,  0.        ,
         0.        ,  0.        ],
       [-0.74699111, -0.92394361, -0.22023669, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.84094839,  2.470391  , -0.22023669, ...,  0.        ,
         0.        ,  0.        ],
       [-0.52014261, -0.64851234,  3.13859829, ...,  0.        ,
         0.        ,  0.        ],
       [-0.06644561,  0.92621781, -1.33984835, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from tensorflow.keras import utils

y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(17376, 3)
(7447, 3)


In [ ]:
print(x_train_transformed.shape)
print(x_test_transformed.shape)
print(y_train.shape)
print(y_test.shape)

(17376, 55)
(7447, 55)
(17376, 3)
(7447, 3)


In [ ]:
def build_hyper_model(hp):
    
    model = keras.Sequential()

    for layer_num in range(hp.Int('num_layers', min_value=1, max_value=3)): 
  
        hp_units = hp.Int('units_' + str(layer_num), min_value=32, max_value=512, step=32)
        hp_activations = hp.Choice('activation_' + str(layer_num), values=['relu', 'elu'])
        model.add(layers.Dense(units = hp_units, activation = hp_activations, kernel_initializer=initializers.he_uniform()))

    model.add(layers.Dense(3, activation='softmax')) 

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = losses.categorical_crossentropy, 
                metrics = ['accuracy'])
                
    
    return model

In [ ]:
tuner = kt.BayesianOptimization(build_hyper_model,
                                objective = 'val_accuracy',
                                max_trials = 10, 
                                directory = 'test_prac_dir', 
                                project_name = 'creditcard_hyper_4')


In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(x_train_transformed, y_train, epochs=20, validation_data = (x_test_transformed, y_test), 
             callbacks=[ClearTrainingOutput()])

Trial 10 Complete [00h 00m 41s]
val_accuracy: 0.6471062302589417

Best val_accuracy So Far: 0.6910164952278137
Total elapsed time: 00h 07m 44s


In [ ]:
#정확도 상위 3개 결과
tuner.results_summary(num_trials=3)

Results summary
Results in test_prac_dir/creditcard_hyper_4
Showing 3 best trials
Trial summary
Hyperparameters:
num_layers: 1
units_0: 32
activation_0: elu
learning_rate: 0.01
units_1: 480
activation_1: relu
units_2: 352
activation_2: relu
Score: 0.6910164952278137
Trial summary
Hyperparameters:
num_layers: 1
units_0: 32
activation_0: relu
learning_rate: 0.01
units_1: 512
activation_1: relu
units_2: 512
activation_2: relu
Score: 0.6899422407150269
Trial summary
Hyperparameters:
num_layers: 1
units_0: 32
activation_0: elu
learning_rate: 0.01
units_1: 32
activation_1: relu
units_2: 512
activation_2: relu
Score: 0.6898079514503479


In [ ]:
#정확도 베스트 모델의 하이퍼파라미터 (idx num==0)
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print("""
The hyperparameter search is complete. 
* Optimal # of layers : {}
* Optimal value of the learning-rate : {}""".format(best_hps.get('num_layers'), best_hps.get('learning_rate')))

for layer_num in range(best_hps.get('num_layers')):
    print('Layer {} - # of Perceptrons :'.format(layer_num), best_hps.get('units_' + str(layer_num)))
    print('Layer {} - Applied activation function :'.format(layer_num), best_hps.get('activation_' + str(layer_num)))


The hyperparameter search is complete. 
* Optimal # of layers : 1
* Optimal value of the learning-rate : 0.01
Layer 0 - # of Perceptrons : 32
Layer 0 - Applied activation function : elu


In [ ]:
#위 결과로 찾은 최적의 하이퍼파라미터를 이용해 모델객체 생성 & 재훈련
model = tuner.hypermodel.build(best_hps)
model.fit(x_train_transformed, y_train, epochs=20, validation_data=(x_test_transformed, y_test))

results = model.evaluate(x_test_transformed, y_test)
print('loss (cross-entropy) :', results[0])
print('Accuracy :', results[1])

Epoch 1/20
543/543 [==============================] - 2s 3ms/step - loss: 0.8894 - accuracy: 0.6412 - val_loss: 0.8635 - val_accuracy: 0.6497
Epoch 2/20
543/543 [==============================] - 2s 3ms/step - loss: 0.8573 - accuracy: 0.6658 - val_loss: 0.8381 - val_accuracy: 0.6791
Epoch 3/20
543/543 [==============================] - 1s 3ms/step - loss: 0.8468 - accuracy: 0.6734 - val_loss: 0.8417 - val_accuracy: 0.6756
Epoch 4/20
543/543 [==============================] - 1s 3ms/step - loss: 0.8449 - accuracy: 0.6754 - val_loss: 0.8311 - val_accuracy: 0.6839
Epoch 5/20
543/543 [==============================] - 1s 3ms/step - loss: 0.8407 - accuracy: 0.6765 - val_loss: 0.8274 - val_accuracy: 0.6783
Epoch 6/20
543/543 [==============================] - 1s 3ms/step - loss: 0.8383 - accuracy: 0.6776 - val_loss: 0.8270 - val_accuracy: 0.6824
Epoch 7/20
543/543 [==============================] - 1s 3ms/step - loss: 0.8366 - accuracy: 0.6774 - val_loss: 0.8307 - val_accuracy: 0.6818
Epoch 